In [ ]:
%pip install tttarena

In [ ]:
import time
import math
from typing import Tuple, List, Optional

import numpy as np

from tttarena.engine.core.engine import TetrisEngine
from tttarena.engine.core.geometry import PIECE_SHAPES
from tttarena.engine.core.exceptions import InvalidMove, NoValidMovesError
from tttarena.simulator.logger import save_log
from tttarena.bots.base_bot import BaseBot

from tttarena.engine.sculptor.metrics import get_height_profile
from tttarena.simulator.runner import SimulationRunner

In [ ]:
def calculate_final_metric(score_S: float, error_A: float, max_possible_score: float) -> float:
    """Вычисляет итоговую метрику на основе счета (S) и ошибки аппроксимации (A)."""
    if score_S < 0 or error_A < 0:
        raise ValueError("Счет и ошибка не могут быть отрицательными.")
    if max_possible_score <= 0:
        raise ValueError("Максимальный счет должен быть положительным.")

    s_norm = min(score_S / max_possible_score, 1.0)
    a_norm = 1.0 / (1.0 + math.sqrt(error_A))

    if s_norm == 0 or a_norm == 0:
        return 0.0

    harmonic_mean = 2 * (s_norm * a_norm) / (s_norm + a_norm)

    return harmonic_mean

## Код бота

Здесь представлен полный код `SimpleBot`. Этот бот использует эвристику, основанную на классических метриках "хорошего" поля в Тетрисе:
- **Совокупная высота:** Штраф за высоту столбиков.
- **"Дыры":** Штраф за пустые клетки, над которыми есть блоки.
- **Неровность:** Штраф за резкие перепады высот между соседними столбиками.

Бот пытается найти такой ход, который минимизирует взвешенную сумму этих метрик. Также он получает большой бонус за очищенные линии.

In [ ]:
class SimpleBot(BaseBot):
    def find_best_move(self, engine: TetrisEngine) -> Tuple[int, int]:
        """Ищет лучший ход для текущей фигуры, используя быструю инкрементальную оценку."""
        best_move: Optional[Tuple[int, int]] = None
        best_score = float("inf")

        current_piece_type = engine.current_piece_type
        if not current_piece_type:
            raise RuntimeError("Попытка найти ход без текущей фигуры.")

        possible_rotations = PIECE_SHAPES[current_piece_type]
        original_height_profile = get_height_profile(engine.board.grid, engine.board.width, engine.board.height)

        for rot_idx, shape in enumerate(possible_rotations):
            for x in range(engine.board.width):
                try:
                    # Быстро находим конечную позицию Y без копирования доски
                    final_y = engine._find_drop_y(shape, x)

                    # Инкрементально вычисляем метрики
                    score, lines_cleared = self._calculate_incremental_score(
                        engine, original_height_profile, shape, x, final_y
                    )

                    score -= lines_cleared * 1000

                    if score < best_score:
                        best_score = score
                        best_move = (x, rot_idx)

                except InvalidMove:
                    continue

        if best_move is None:
            # Если ни один ход не найден (например, все приводят к немедленному проигрышу),
            # пытаемся найти хоть какой-то валидный ход, даже если он плохой.
            # Это запасной вариант, чтобы избежать NoValidMovesError.
            for rot_idx in range(len(possible_rotations)):
                for x in range(engine.board.width):
                    try:
                        # Просто проверим, можно ли вообще сбросить фигуру
                        engine._find_drop_y(possible_rotations[rot_idx], x)
                        return (x, rot_idx) # Возвращаем первый же валидный ход
                    except InvalidMove:
                        continue
            
            raise NoValidMovesError(
                "Не найдено ни одного валидного хода для текущей фигуры."
            )

        return best_move

    def _calculate_incremental_score(
        self, engine: TetrisEngine, original_height_profile: List[int],
        shape: Tuple[Tuple[int, int], ...], x: int, y: int
    ) -> Tuple[float, int]:
        """
        Вычисляет "штраф" для хода, не создавая новую доску.
        Возвращает (оценка, количество очищенных линий).
        """
        board = engine.board
        temp_height_profile = list(original_height_profile)
        
        piece_coords = set()
        for px, py in shape:
            abs_x, abs_y = x + px, y + py
            if not (0 <= abs_x < board.width and 0 <= abs_y < board.height):
                raise InvalidMove("Фигура выходит за границы доски")
            if board.grid[abs_y, abs_x] != 0:
                raise InvalidMove("Столкновение с существующим блоком")
            
            piece_coords.add((abs_x, abs_y))
            new_height = board.height - abs_y
            if new_height > temp_height_profile[abs_x]:
                temp_height_profile[abs_x] = new_height

        lines_cleared = 0
        cleared_rows = set()
        unique_y_coords = sorted(list(set(abs_y for _, abs_y in piece_coords)))
        
        for row_y in unique_y_coords:
            is_line_full = True
            for col_x in range(board.width):
                if board.grid[row_y, col_x] == 0 and (col_x, row_y) not in piece_coords:
                    is_line_full = False
                    break
            if is_line_full:
                lines_cleared += 1
                cleared_rows.add(row_y)

        holes = 0
        for abs_x, col_height in enumerate(temp_height_profile):
            start_y = board.height - col_height
            for row_y in range(start_y, board.height):
                if board.grid[row_y, abs_x] == 0 and (abs_x, row_y) not in piece_coords:
                    if row_y not in cleared_rows:
                        holes += 1

        bumpiness = np.sum(np.abs(np.diff(temp_height_profile)))
        aggregate_height = np.sum(temp_height_profile)

        # Веса из оригинальной статьи Pierre Dellacherie
        # https://github.com/JiangkaiWu/AI-Tetris/blob/master/ai.cpp
        score = aggregate_height * 0.51 + holes * 0.76 + bumpiness * 0.18

        return score, lines_cleared


## Запуск симуляции

In [ ]:
SEED = 42 

my_bot = SimpleBot()
engine = TetrisEngine(width=10, height=20, seed=SEED)
runner = SimulationRunner(engine, my_bot)

start_time = time.time()
results = runner.run(start_time=start_time)
duration = time.time() - start_time
save_log(results, SEED)

Игра окончена: Невозможно разместить новую фигуру 'T'. Игра окончена.


In [8]:
print("\n" + "=" * 20 + " ИТОГОВЫЕ РЕЗУЛЬТАТЫ " + "=" * 20)
print(f"Продолжительность: {duration:.2f} сек.")
print(f"Сид: {results['seed']}")
print(f"Итоговый счет (S): {results['final_score_S']}")
print(f"Ошибка аппроксимации (A): {results['final_error_A']:.4f}")
print(f"ФИНАЛЬНАЯ МЕТРИКА: {results['final_metric']:.4f}")
print(f"RPS (фигур в секунду): {results['final_rps']:.2f}")
print("=" * 62)


==================== ИТОГОВЫЕ РЕЗУЛЬТАТЫ ====================
Продолжительность: 1.89 сек.
Сид: 42
Итоговый счет (S): 113700
Ошибка аппроксимации (A): 35.9376
ФИНАЛЬНАЯ МЕТРИКА: 0.2285
RPS (фигур в секунду): 1395.68
